In [5]:
import pandas as pd
from sodapy import Socrata
import openmeteo_requests
import requests_cache
from retry_requests import retry

OPEN-METEO

In [7]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": ["temperature_2m", "relative_humidity_2m", "rain", "shortwave_radiation"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["rain"] = hourly_rain
hourly_data["shortwave_radiation"] = hourly_shortwave_radiation

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe


Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m,relative_humidity_2m,rain,shortwave_radiation
0,2024-06-21 00:00:00+00:00,17.295500,76.0,0.0,0.0
1,2024-06-21 01:00:00+00:00,17.045500,76.0,0.0,0.0
2,2024-06-21 02:00:00+00:00,16.695499,76.0,0.0,0.0
3,2024-06-21 03:00:00+00:00,16.245501,80.0,0.0,0.0
4,2024-06-21 04:00:00+00:00,16.245501,81.0,0.0,32.0
...,...,...,...,...,...
163,2024-06-27 19:00:00+00:00,25.319500,40.0,0.0,83.0
164,2024-06-27 20:00:00+00:00,23.469501,45.0,0.0,11.0
165,2024-06-27 21:00:00+00:00,21.869501,49.0,0.0,0.0
166,2024-06-27 22:00:00+00:00,20.719501,53.0,0.0,0.0


In [8]:
hourly_dataframe.to_csv('hourly_weather_data.csv', index=False)


IDEAM

In [10]:


client = Socrata("www.datos.gov.co", None)

results = client.get("uext-mhny", limit=2000)

results_df = pd.DataFrame.from_records(results)

results_df

,codigoestacion,codigosensor,fechaobservacion,valorobservado,nombreestacion,departamento,municipio,zonahidrografica,latitud,longitud,descripcionsensor,unidadmedida
0,0021235030,0027,2007-06-06T16:00:00.000,49,UNIVERSIDAD DE CUNDINAMARCA - AUT,CUNDINAMARCA,GIRARDOT,ALTO MAGDALENA,4.305333333,-74.80811111,Humedad del aire 2 mt,%
1,0053115501,0027,2017-11-05T18:50:00.000,98.93397,DAGUA - AUT,VALLE DEL CAUCA,DAGUA,TAPAJE - DAGUA - DIRECTOS,3.610441667,-76.63785833,Humedad del aire 2 mt,%
2,0026125508,0027,2019-02-21T21:20:00.000,85.54683,LACATALINA - AUT,RISARALDA,PEREIRA,CAUCA,4.7475,-75.73805556,Humedad del aire 2 mt,%
3,0052015501,0027,2016-07-07T10:00:00.000,46,BALBOA - AUT,CAUCA,BALBOA,PATÍA,2.125191667,-77.19266667,Humedad del aire 2 mt,%
4,0023035030,0027,2005-11-02T18:00:00.000,73,BASE PALANQUERO - AUT,CUNDINAMARCA,PUERTO SALGAR,MEDIO MAGDALENA,5.492861111,-74.658,Humedad del aire 2 mt,%
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0029065501,0027,2018-02-02T09:45:00.000,47.21194,"CIENAGA,LA - AUT",MAGDALENA,CIÉNAGA,BAJO MAGDALENA,10.89785833,-74.08156667,Humedad del aire 2 mt,%
1996,0026145508,0027,2016-07-03T05:25:00.000,100,SAN JOSE - AUT,CALDAS,SAN JOSE,CAUCA,5.094972222,-75.79174722,Humedad del aire 2 mt,%
1997,0021195502,0027,2015-06-05T12:00:00.000,99,AGUAS CLARAS - CAR,CUNDINAMARCA,SAN BERNARDO,ALTO MAGDALENA,4.145,-74.428,Humedad del aire 2 mt,%
1998,0035185010,0027,2010-08-15T06:00:00.000,100,LA PALOMERA - AUT,META,PUERTO LÓPEZ,META,4.260361111,-72.56447222,Humedad del aire 2 mt,%


In [11]:
results_df.to_csv('results_df.csv', index=False)
